In [1]:
import pandas as pd
import transformers
import torch
from datasets import load_dataset

/home/oumar/Projects/venv/lib64/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
dataset = load_dataset("b-mc2/sql-create-context", split="train")

# split the dataset into training, validation and test
dataset_test = dataset.train_test_split(test_size=0.2) 

# split the dataset into training and validation
dataset = dataset_test['train'].train_test_split(test_size=0.2)

In [11]:
dataset['train'][0]

{'answer': 'SELECT power FROM table_name_28 WHERE model = "2.0 tdi (cr) dpf" AND years = "2010–2011"',
 'question': 'What is the power for model  2.0 tdi (cr) dpf, and a Years of 2010–2011?',
 'context': 'CREATE TABLE table_name_28 (power VARCHAR, model VARCHAR, years VARCHAR)'}

## Mistral 7b

## Preprocessing

In [10]:
# forma data 
system_message = """You are a natural language to sql query translator model. Users will ask you a question in English and you will generate a SQL query based on the table provided: {table}"""

def format_data(dataset):
    return {
        "messages": [
            {"role": "system", "content": system_message.format(table=dataset["context"])},
            {"role": "user", "content": dataset["question"]},
            {"role": "assistant", "content": dataset["answer"]}
        ]}


train_data = dataset['train'].map(format_data)
val_data = dataset['test'].map(format_data)
test_data = dataset_test['test'].map(format_data)

Map:   0%|          | 0/56574 [00:00<?, ? examples/s]

Map: 100%|██████████| 15716/15716 [00:02<00:00, 6377.88 examples/s]


In [16]:
# train_data = train_data.map(lambda x: {"schema": format_schema(x["table"])})
train_data["messages"][0]

[{'content': 'You are a natural language to sql query translator model. Users will ask you a question in English and you will generate a SQL query based on the table provided: CREATE TABLE table_name_28 (power VARCHAR, model VARCHAR, years VARCHAR)',
  'role': 'system'},
 {'content': 'What is the power for model  2.0 tdi (cr) dpf, and a Years of 2010–2011?',
  'role': 'user'},
 {'content': 'SELECT power FROM table_name_28 WHERE model = "2.0 tdi (cr) dpf" AND years = "2010–2011"',
  'role': 'assistant'}]

In [12]:
df = pd.DataFrame(train_data)
df2 = pd.DataFrame(val_data)
df3 = pd.DataFrame(test_data)

In [13]:
train_data = df[["messages"]]
val_data = df2[["messages"]]
test_data = df3[["messages"]]

# convert to Dataset
from datasets import Dataset
train_data = Dataset.from_pandas(train_data)
val_data = Dataset.from_pandas(val_data)
test_data = Dataset.from_pandas(test_data)

train_data[0]

{'messages': [{'content': "You are a natural language to sql query translator model. Users will ask you a question in English and you will generate a SQL query based on the table provided: {'header': ['State/territory', 'Text/background colour', 'Format', 'Current slogan', 'Current series', 'Notes'], 'page_title': '', 'page_id': '', 'types': ['text', 'text', 'text', 'text', 'text', 'text'], 'id': '1-1000181-1', 'section_title': '', 'caption': '', 'rows': [['Australian Capital Territory', 'blue/white', 'Yaa·nna', 'ACT · CELEBRATION OF A CENTURY 2013', 'YIL·00A', 'Slogan screenprinted on plate'], ['New South Wales', 'black/yellow', 'aa·nn·aa', 'NEW SOUTH WALES', 'BX·99·HI', 'No slogan on current series'], ['New South Wales', 'black/white', 'aaa·nna', 'NSW', 'CPX·12A', 'Optional white slimline series'], ['Northern Territory', 'ochre/white', 'Ca·nn·aa', 'NT · OUTBACK AUSTRALIA', 'CB·06·ZZ', 'New series began in June 2011'], ['Queensland', 'maroon/white', 'nnn·aaa', 'QUEENSLAND · SUNSHINE S

In [35]:
import json
def open_json(file):
    with open(file, "r") as f:
        data = f.readlines()
    return data

In [39]:
train_data = open_json("train_data.jsonl")
val_data = open_json("val_data.jsonl")
test_data = open_json("test_data.jsonl")
print(json.loads(train_data[0]))

{'messages': [{'content': "You are a natural language to sql query translator model. Users will ask you a question in English and you will generate a SQL query based on the table provided: {'header': ['State/territory', 'Text/background colour', 'Format', 'Current slogan', 'Current series', 'Notes'], 'page_title': '', 'page_id': '', 'types': ['text', 'text', 'text', 'text', 'text', 'text'], 'id': '1-1000181-1', 'section_title': '', 'caption': '', 'rows': [['Australian Capital Territory', 'blue/white', 'Yaa·nna', 'ACT · CELEBRATION OF A CENTURY 2013', 'YIL·00A', 'Slogan screenprinted on plate'], ['New South Wales', 'black/yellow', 'aa·nn·aa', 'NEW SOUTH WALES', 'BX·99·HI', 'No slogan on current series'], ['New South Wales', 'black/white', 'aaa·nna', 'NSW', 'CPX·12A', 'Optional white slimline series'], ['Northern Territory', 'ochre/white', 'Ca·nn·aa', 'NT · OUTBACK AUSTRALIA', 'CB·06·ZZ', 'New series began in June 2011'], ['Queensland', 'maroon/white', 'nnn·aaa', 'QUEENSLAND · SUNSHINE S

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, AdamW, BitsAndBytesConfig, get_scheduler, DataCollatorWithPadding, AutoModelForSeq2SeqLM
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model
import torch
from accelerate import Accelerator, init_empty_weights, infer_auto_device_map

# enable distributed training
accelerate = Accelerator()

checkpoint = "mistralai/Mistral-7B-Instruct-v0.3"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

# OOV token
tokenizer.pad_token = tokenizer.unk_token
tokenizer.pad_token_id = tokenizer.unk_token_id

# padding side
tokenizer.pad_padding_side = "right"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

device_map = {"":0}

with init_empty_weights():
    model = AutoModelForCausalLM.from_pretrained(checkpoint, quantization_config=bnb_config, device_map={"":0})
# device_map = infer_auto_device_map(model, max_memory={"cuda": "2GiB", "cpu": "16GiB"}) 
# model = model.to(device_map)
model = get_peft_model(model, bnb_config)



In [ ]:
# gradient checkpointing
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [ ]:
# parameters
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [ ]:
# Lora config
lora_config = LoraConfig(
    r = 256,
    lora_alpha=128,
    target_modules="all-linear",
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)
print_trainable_parameters(model)

In [ ]:
layers = model.state_dict().keys()
for name in layers:
    print(name)

In [ ]:
from torch.utils.data import DataLoader

def tokenize_function(dataset):
    return tokenizer(dataset["messages"], return_tensors="pt")

train_data = train_data.map(tokenize_function, batched=True)
val_data = val_data.map(tokenize_function, batched=True)
test_data = test_data.map(tokenize_function, batched=True)

train_dataloader = DataLoader(train_data, batch_size=8, shuffle=True)
val_dataloader = DataLoader(val_data, batch_size=8, shuffle=False)
test_dataloader = DataLoader(test_data, batch_size=8, shuffle=False)

data_collator = DataCollatorWithPadding(tokenizer)

## Training

In [10]:
# Evaluation
raw_datasets = load_dataset("glue", "mrpc")
raw_datasets["train"][0]


{'sentence1': 'Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .',
 'sentence2': 'Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .',
 'label': 1,
 'idx': 0}

In [ ]:
from tqdm.auto import tqdm


optimizer = AdamW(model.parameters(), lr=5e-5)
num_epochs = 50
num_train_steps = len(train_data) * num_epochs

train_dataloader, val_dataloader, model, optimizer = accelerate.prepare(train_dataloader, val_dataloader, model, optimizer)

lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_train_steps,
    gradient_accumulation_steps=4
)

progress_bar = tqdm(range(num_train_steps))

model.train()
losses = []
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device_map) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

        losses.append(loss.item())

# save model
model.save_pretrained("natural_language_to_sql")

        

In [ ]:
# test model
model_predict = "natural_langauge_to_sql"

model = AutoModelForCausalLM.from_pretrained(model_predict, torch_dtype=torch.float16)
